In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import pandas as pd
import numpy as np
from scipy import stats

def flag_statistical_outliers(df, threshold=3):
    """Flags cells where |Z-Score| > threshold with improved error handling."""
    for col in df.select_dtypes(include=np.number).columns:
        try:
            # Only calculate Z-scores for non-null values
            non_null = df[col].dropna()
            if len(non_null) > 1:  # Need at least 2 values to calculate Z-score
                z_scores = np.abs(stats.zscore(non_null))
                # Initialize with 0 and only set 1 where condition is True
                df[f"{col}_Is_Outlier"] = 0
                df.loc[non_null.index, f"{col}_Is_Outlier"] = (z_scores > threshold).astype(int)
        except Exception as e:
            print(f"Could not process column {col}: {str(e)}")
            continue
    return df

def flag_rare_categories(df, threshold=0.01):
    """Flags cells with categories appearing in < threshold of rows."""
    for col in df.select_dtypes(include=['object', 'category']).columns:
        try:
            value_counts = df[col].value_counts(normalize=True)
            rare_categories = value_counts[value_counts < threshold].index
            df[f"{col}_Is_Rare"] = df[col].isin(rare_categories).astype(int)
        except Exception as e:
            print(f"Could not process column {col}: {str(e)}")
            continue
    return df

def highlight_cells_in_excel(file_path, df, data_start_row=4):
    try:
        wb = load_workbook(file_path)
        ws = wb.active

        outlier_fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid') #Red for Z score outliers
        rare_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid') # yellow for rare categoricals

        for col in df.columns:
            if "_Is_Outlier" in col:
                original_col = col.replace("_Is_Outlier", "")
                if original_col in df.columns:
                    col_idx = df.columns.get_loc(original_col) + 1
                    for i, is_outlier in enumerate(df[col], start=data_start_row-1):
                        if is_outlier == 1:
                            ws.cell(row=i+1, column=col_idx).fill = outlier_fill

            if "_Is_Rare" in col:
                original_col = col.replace("_Is_Rare", "")
                if original_col in df.columns:
                    col_idx = df.columns.get_loc(original_col) + 1
                    for i, is_rare in enumerate(df[col], start=data_start_row-1):
                        if is_rare == 1:
                            ws.cell(row=i+1, column=col_idx).fill = rare_fill

        output_file_path = filedialog.asksaveasfilename(
            defaultextension=".xlsx",
            title="Save the highlighted Excel file"
        )
        if output_file_path:
            wb.save(output_file_path)
            messagebox.showinfo("Success", f"File saved: {output_file_path}")
    except Exception as e:
        messagebox.showerror("Error", f"Failed to highlight file: {str(e)}")

def process_file():
    file_path = filedialog.askopenfilename(
        title="Select Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    if not file_path:
        return

    try:
        # Read Excel file with proper handling of headers and data start row
        df = pd.read_excel(file_path, skiprows=2, header=0)
        
        if df.empty:
            messagebox.showerror("Error", "The selected file contains no data")
            return

        # Process data
        df = flag_statistical_outliers(df)
        df = flag_rare_categories(df)

        # Highlight results
        highlight_cells_in_excel(file_path, df, data_start_row=4)
    except Exception as e:
        messagebox.showerror("Error", f"Failed to process file: {str(e)}")

# GUI Setup
root = tk.Tk()
root.title("Excel Data Quality Checker")
tk.Button(root, text="Select Excel File", command=process_file, padx=20, pady=10).pack(pady=20)
root.mainloop()